# Original Data Import and Merging

Same as in 01_data_combine_and_clean but with filtered transaction data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

# Orders and Accounts

In [2]:
#Import in data

order_df = pd.read_csv("./csv_files/order.csv") 

account_df = pd.read_csv("./csv_files/account.csv")

account_df.head()

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [3]:
order_df.shape

(6471, 6)

In [4]:
#Fill in Nan's with Others

order_df['k_symbol'] = order_df['k_symbol'].fillna('Other')
order_df['k_symbol'].value_counts(dropna=False)
order_df.head(10)

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,Other
5,29406,3,AB,59972357,3539.0,POJISTNE
6,29407,4,UV,26693541,2078.0,SIPO
7,29408,4,UV,5848086,1285.0,SIPO
8,29409,5,GH,37390208,2668.0,SIPO
9,29410,6,AB,44486999,3954.0,SIPO


In [5]:
# Get the dummies
order_dum = pd.get_dummies(order_df[['k_symbol']])

#COmbine dummies variables with original data
order_all = pd.concat([order_df,order_dum],axis=1,ignore_index=False)  

# Drop uneeded columns
order_all = order_all.drop(columns = ['order_id','bank_to','account_to','k_symbol'])

# Multiply dummies by amounts
for col in order_dum.columns:
    order_all[col] = order_all[col] * order_all['amount']

# Grouby by account id
order_acc = order_all.groupby('account_id').sum()

# Rename columns to avoid later confusion
order_acc = order_acc.rename(columns={"amount": "amount_order"})

order_acc.head()

,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER
account_id,,,,,,
1,2452.0,0.0,0.0,0.0,2452.0,0.0
2,10638.7,0.0,0.0,0.0,7266.0,3372.7
3,5001.0,0.0,327.0,3539.0,1135.0,0.0
4,3363.0,0.0,0.0,0.0,3363.0,0.0
5,2668.0,0.0,0.0,0.0,2668.0,0.0


In [6]:
account_df.head(10)

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30
5,6,51,POPLATEK MESICNE,1994-09-27
6,7,60,POPLATEK MESICNE,1996-11-24
7,8,57,POPLATEK MESICNE,1995-09-21
8,9,70,POPLATEK MESICNE,1993-01-27
9,10,54,POPLATEK MESICNE,1996-08-28


In [7]:
# Join orders onto accounts

account_df = pd.merge(account_df, order_acc, on='account_id', how='left')

account_df.head()

#Order now done.... onto to transactions

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0


# Transactions

In [8]:
# First half of transactions
# Read in data
trans_1_df = pd.read_csv("./csv_files/trans_1_filter.csv",parse_dates=[2])

# Drop unused columns
trans_1_df = trans_1_df.drop(columns=['trans_id','bank','account'])

# Fill emptys with nan's
trans_1_df['k_symbol'] = trans_1_df['k_symbol'].replace(r'^\s*$', np.nan, regex=True)

# Fill in undefined operations as other
trans_1_df['operation'] = trans_1_df['operation'].fillna('Other')
trans_1_df['k_symbol'] = trans_1_df['k_symbol'].fillna('Other')

# Get the Dummies
trans_1_df_dum = pd.get_dummies(trans_1_df[['type','operation','k_symbol']])

#Combine with original data
trans_1_df_all = pd.concat([trans_1_df,trans_1_df_dum],axis=1,ignore_index=False)  

# Muliply dummies by thier value
for col in trans_1_df_dum.columns:
    trans_1_df_all[col] = trans_1_df_all[col] * trans_1_df_all['amount']

# Drop uneeded columns
trans_1_df_all = trans_1_df_all.drop(columns = ['type','operation','k_symbol'])    

trans_1_df_all.head()


,account_id,date,amount,balance,type_PRIJEM,type_VYBER,type_VYDAJ,operation_Other,operation_PREVOD NA UCET,operation_PREVOD Z UCTU,operation_VKLAD,operation_VYBER,operation_VYBER KARTOU,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SANKC. UROK,k_symbol_SIPO,k_symbol_SLUZBY,k_symbol_UROK
0,2,1993-02-26,1100,1100,1100,0,0,0,0,0,1100,0,0,1100,0,0,0,0,0
1,2,1993-03-12,20236,21336,20236,0,0,0,0,20236,0,0,0,20236,0,0,0,0,0
2,2,1993-04-12,20236,45286,20236,0,0,0,0,20236,0,0,0,20236,0,0,0,0,0
3,2,1993-05-12,20236,54631,20236,0,0,0,0,20236,0,0,0,20236,0,0,0,0,0
4,2,1993-06-12,30354,67530,30354,0,0,0,0,30354,0,0,0,30354,0,0,0,0,0


In [9]:
# Groupby account id and do some aggregations

trans_acc = trans_1_df_all.groupby('account_id').agg({ 'date' : ['min','max'],
                            'amount' : ['sum','mean','std'],
                            'balance': ['min','max','mean','std'],
                            'type_PRIJEM' : ['sum'],
                            'type_VYBER': ['sum'],
                            'type_VYDAJ': ['sum'], 
                            'operation_Other': ['sum'], 
                            'operation_PREVOD NA UCET': ['sum'],
                            'operation_PREVOD Z UCTU': ['sum'], 
                            'operation_VKLAD': ['sum'],
                            'operation_VYBER': ['sum'],
                            'operation_VYBER KARTOU': ['sum'], 
                            #'k_symbol_DUCHOD': ['sum'],
                            'k_symbol_Other': ['sum'], 
                            'k_symbol_POJISTNE': ['sum'], 
                            'k_symbol_SANKC. UROK': ['sum'],
                            'k_symbol_SIPO': ['sum'], 
                            'k_symbol_SLUZBY': ['sum'], 
                            'k_symbol_UROK': ['sum'], 
                            #'k_symbol_UVER': ['sum']
                            })


# Fix the column names
trans_acc.columns = ["_".join(x) for x in trans_acc.columns.ravel()]

trans_acc.head()

,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum
account_id,,,,,,,,,,,,,,,,,,,,,,,,
2,1993-02-26,1993-12-31,429534,7954.333333,8793.201591,1100,67530,32590.759259,12061.802206,228687,47827,153020,1291,36330,222596,4800,164517,0,391823,0,0,36330,90,1291
19,1995-04-07,1996-04-10,468508,5856.350000,7074.850795,715,58158,25197.137500,15039.321182,242180,28308,198020,2835,0,211945,27400,226328,0,338153,0,0,127400,120,2835
25,1996-07-28,1997-12-06,1986693,12113.981707,15599.382022,900,119653,62991.408537,22476.380134,1009156,66843,910694,4087,114384,0,1005069,863153,0,1884662,2296,0,95438,210,4087
37,1997-08-18,1998-10-12,878356,7572.034483,11817.595990,-1011,104762,39954.034483,23451.696220,447360,41655,389341,2303,59976,0,445057,371020,0,846920,1164,0,27804,165,2303
38,1997-08-08,1998-04-10,259391,4716.200000,5740.513545,13841,55991,31383.581818,10950.723180,148404,5896,105091,1114,29671,131490,15800,81316,0,229641,912,0,27604,120,1114


In [10]:
#Second Half of Transactions

# Do the same as above but for 2nd half of transactions

trans_2_df = pd.read_csv("./csv_files/trans_2_filter.csv",parse_dates=[2])

trans_2_df = trans_2_df.drop(columns=['trans_id','bank','account'])

trans_2_df['k_symbol'] = trans_2_df['k_symbol'].replace(r'^\s*$', np.nan, regex=True)

trans_2_df['operation'] = trans_2_df['operation'].fillna('Other')
trans_2_df['k_symbol'] = trans_2_df['k_symbol'].fillna('Other')

trans_2_df_dum = pd.get_dummies(trans_2_df[['type','operation','k_symbol']])

trans_2_df_all = pd.concat([trans_2_df,trans_2_df_dum],axis=1,ignore_index=False)  

for col in trans_2_df_dum.columns:
    trans_2_df_all[col] = trans_2_df_all[col] * trans_2_df_all['amount']

trans_2_df_all = trans_2_df_all.drop(columns = ['type','operation','k_symbol'])    

trans_2_acc = trans_2_df_all.groupby('account_id').agg({ 'date' : ['min','max'],
                            'amount' : ['sum','mean','std'],
                            'balance': ['min','max','mean','std'],
                            'type_PRIJEM' : ['sum'],
                            'type_VYBER': ['sum'],
                            'type_VYDAJ': ['sum'], 
                            'operation_Other': ['sum'], 
                            'operation_PREVOD NA UCET': ['sum'],
                            'operation_PREVOD Z UCTU': ['sum'], 
                            'operation_VKLAD': ['sum'],
                            'operation_VYBER': ['sum'],
                            'operation_VYBER KARTOU': ['sum'], 
                            #'k_symbol_DUCHOD': ['sum'],
                            'k_symbol_Other': ['sum'], 
                            'k_symbol_POJISTNE': ['sum'], 
                            'k_symbol_SANKC. UROK': ['sum'],
                            'k_symbol_SIPO': ['sum'], 
                            'k_symbol_SLUZBY': ['sum'], 
                            'k_symbol_UROK': ['sum'], 
                            #'k_symbol_UVER': ['sum']
                            })



trans_2_acc.columns = ["_".join(x) for x in trans_2_acc.columns.ravel()]

trans_2_acc.head()

,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum
account_id,,,,,,,,,,,,,,,,,,,,,,,,
2322,1995-01-02,1996-06-25,1435446,8064.303371,13242.463681,900,119893,43657.876404,20752.096898,729496,30430,675520,2856,106880,0,726640,599070,0,1362302,16304,0,53744,240,2856
2334,1996-03-19,1997-07-31,532508,5664.978723,5431.842920,600,53030,31897.574468,9079.283030,286844,0,245664,1979,128484,211365,73500,117180,0,401865,0,0,128484,180,1979
2335,1996-04-04,1997-11-10,618737,3555.959770,5267.722539,-17030,51161,12006.442529,14745.181621,312547,0,306190,2619,78990,0,309928,227200,0,577723,7275,355,30540,225,2619
2343,1994-12-09,1995-10-31,226118,4348.423077,4545.657011,300,46568,27019.634615,7892.751221,128470,0,97648,1108,28338,0,127362,69310,0,196582,0,0,28338,90,1108
2345,1993-10-25,1994-11-30,544164,7353.567568,8335.592277,500,62239,33658.743243,11909.096066,282042,38279,223843,1675,32508,279567,800,229614,0,509846,0,0,32508,135,1675


In [11]:
# Join the two transaction tables together
trans_all = pd.concat([trans_acc,trans_2_acc],axis=0,ignore_index=False)  

# Group by index to only take owner related info 
trans_all = trans_all.groupby(trans_all.index).first()

# Reset Index
trans_all = trans_all.reset_index()

# Change account id type to allow later joins
trans_all['account_id'] = trans_all['account_id'].astype('int64')

trans_all.head(10)

,account_id,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum
0,2,1993-02-26,1993-12-31,429534,7954.333333,8793.201591,1100,67530,32590.759259,12061.802206,228687,47827,153020,1291,36330,222596,4800,164517,0,391823,0,0,36330,90,1291
1,19,1995-04-07,1996-04-10,468508,5856.350000,7074.850795,715,58158,25197.137500,15039.321182,242180,28308,198020,2835,0,211945,27400,226328,0,338153,0,0,127400,120,2835
2,25,1996-07-28,1997-12-06,1986693,12113.981707,15599.382022,900,119653,62991.408537,22476.380134,1009156,66843,910694,4087,114384,0,1005069,863153,0,1884662,2296,0,95438,210,4087
3,37,1997-08-18,1998-10-12,878356,7572.034483,11817.595990,-1011,104762,39954.034483,23451.696220,447360,41655,389341,2303,59976,0,445057,371020,0,846920,1164,0,27804,165,2303
4,38,1997-08-08,1998-04-10,259391,4716.200000,5740.513545,13841,55991,31383.581818,10950.723180,148404,5896,105091,1114,29671,131490,15800,81316,0,229641,912,0,27604,120,1114
5,67,1994-10-19,1996-04-30,1602800,12822.400000,16442.490637,700,107070,52523.336000,20955.670441,813242,39016,750542,3487,95732,809055,700,693826,0,1509601,0,0,89502,210,3487
6,97,1996-05-05,1997-08-07,571601,4844.076271,6935.994091,900,60335,33057.457627,12670.964436,306057,0,265544,2245,41079,0,303812,224465,0,554798,33,0,14360,165,2245
7,103,1996-03-10,1997-11-30,716640,8634.216867,12214.959607,379,116038,46266.578313,28827.023350,368308,101793,246539,3655,0,0,364653,348332,0,712846,0,19,0,120,3655
8,105,1997-07-10,1998-12-02,480620,8582.500000,11708.468599,800,74694,30797.535714,16415.423817,255396,4019,221205,2563,0,0,252833,225224,0,477952,0,0,0,105,2563
9,110,1996-07-17,1997-09-06,608234,7240.880952,9829.957741,900,79652,46148.273810,12673.310171,331885,0,276349,2353,28234,0,329532,248115,0,578296,0,0,27450,135,2353


In [12]:
# Join transactions onto our account and order info
account_df = pd.merge(account_df, trans_all, on='account_id', how='left')

account_df.head()

#Transactions added

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1993-12-31,429534.0,7954.333333,8793.201591,1100.0,67530.0,32590.759259,12061.802206,228687.0,47827.0,153020.0,1291.0,36330.0,222596.0,4800.0,164517.0,0.0,391823.0,0.0,0.0,36330.0,90.0,1291.0
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Districts

In [13]:
# Read in districts 
dist_df = pd.read_csv("./csv_files/district.csv")

#Drop unused columns
dist_df = dist_df.drop(columns = ['A2','A3']) 

dist_df.head()



,district_id,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,2,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674
2,3,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813
3,4,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892
4,5,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040


In [14]:
# Merge onto accounts, orders, etc
account_df = pd.merge(account_df, dist_df, on='district_id', how='left')

account_df.head()

#District Complete, now merge card and client onto disp

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1993-12-31,429534.0,7954.333333,8793.201591,1100.0,67530.0,32590.759259,12061.802206,228687.0,47827.0,153020.0,1291.0,36330.0,222596.0,4800.0,164517.0,0.0,391823.0,0.0,0.0,36330.0,90.0,1291.0,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107870,84,29,6,1,6,58.0,8754,3.8,4.31,137,3804.0,3868
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58796,22,16,7,1,5,51.9,9045,3.1,3.60,124,1845.0,1879


# Client, Card and Dispositions

In [15]:
#Read in files
disp_df = pd.read_csv("./csv_files/disp.csv")
client_df = pd.read_csv("./csv_files/client.csv")
card_df = pd.read_csv("./csv_files/card.csv",parse_dates=[3])

card_df.head()

,card_id,disp_id,type,issued
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [16]:
# Join Client info onto Dispositions
disp_df = pd.merge(disp_df, client_df, on='client_id',how='left')

# Join Credit Card Data onto Dispositions
disp_df = pd.merge(disp_df, card_df, on='disp_id',how='left')

# Group by Account_id
disp_df_all = disp_df.groupby('account_id').first()

# Drop useless columns
disp_df_all = disp_df_all.drop(columns = ['disp_id','client_id','district_id','card_id']) 

disp_df_all = disp_df_all.reset_index()

disp_df_all['account_id'] = disp_df_all['account_id'].astype('int64')

# If they don't have a credit card, set to no card
disp_df_all['type_y'] = disp_df_all['type_y'].fillna('No Card')

disp_df_all.head()

,account_id,type_x,gender,birth_date,type_y,issued
0,1,OWNER,F,1970-12-13,No Card,NaT
1,2,OWNER,M,1945-02-04,No Card,NaT
2,3,OWNER,M,1956-12-01,No Card,NaT
3,4,OWNER,M,1919-09-22,No Card,NaT
4,5,OWNER,M,1929-01-25,No Card,NaT


In [17]:
# Merge dispositions, cleint and card info onto all other info
account_df_fin = pd.merge(account_df, disp_df_all, on='account_id',how='left')

# Output all info to be used by our models
account_df_fin.to_csv('./csv_files/accounts_merged_filtered_transactions.csv',index=False)

account_df_fin.head()

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,type_x,gender,birth_date,type_y,issued
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,OWNER,F,1970-12-13,No Card,NaT
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1993-12-31,429534.0,7954.333333,8793.201591,1100.0,67530.0,32590.759259,12061.802206,228687.0,47827.0,153020.0,1291.0,36330.0,222596.0,4800.0,164517.0,0.0,391823.0,0.0,0.0,36330.0,90.0,1291.0,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,OWNER,M,1945-02-04,No Card,NaT
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040,OWNER,M,1956-12-01,No Card,NaT
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107870,84,29,6,1,6,58.0,8754,3.8,4.31,137,3804.0,3868,OWNER,M,1919-09-22,No Card,NaT
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58796,22,16,7,1,5,51.9,9045,3.1,3.60,124,1845.0,1879,OWNER,M,1929-01-25,No Card,NaT


In [18]:
account_df.shape

(4500, 47)

# Loans

In [19]:
#Merge onto our loans
loans_df = pd.read_csv("./csv_files/loan.csv",parse_dates=[2])
loans_base_df = pd.merge(loans_df, account_df_fin, on='account_id',how='left')

# Output the final dataset to be used by our models later
loans_base_df.to_csv('./csv_files/loans_merged_filtered_transactions.csv',index=False)

loans_base_df.head()

,loan_id,account_id,date_x,amount,duration,payments,status,district_id,frequency,date_y,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,type_x,gender,birth_date,type_y,issued
0,4959,2,1994-01-05,80952,24,3373.0,A,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1993-12-31,429534.0,7954.333333,8793.201591,1100.0,67530.0,32590.759259,12061.802206,228687.0,47827.0,153020.0,1291.0,36330.0,222596.0,4800.0,164517.0,0.0,391823.0,0.0,0.0,36330.0,90.0,1291.0,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,OWNER,M,1945-02-04,No Card,NaT
1,4961,19,1996-04-29,30276,12,2523.0,B,21,POPLATEK MESICNE,1995-04-07,2523.2,0.0,0.0,0.0,0.0,2523.2,1995-04-07,1996-04-10,468508.0,5856.350000,7074.850795,715.0,58158.0,25197.137500,15039.321182,242180.0,28308.0,198020.0,2835.0,0.0,211945.0,27400.0,226328.0,0.0,338153.0,0.0,0.0,127400.0,120.0,2835.0,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354,OWNER,F,1939-04-23,No Card,NaT
2,4962,25,1997-12-08,30276,12,2523.0,A,68,POPLATEK MESICNE,1996-07-28,10614.2,0.0,1110.0,164.0,6817.0,2523.2,1996-07-28,1997-12-06,1986693.0,12113.981707,15599.382022,900.0,119653.0,62991.408537,22476.380134,1009156.0,66843.0,910694.0,4087.0,114384.0,0.0,1005069.0,863153.0,0.0,1884662.0,2296.0,0.0,95438.0,210.0,4087.0,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887,OWNER,M,1962-02-09,No Card,NaT
3,4967,37,1998-10-14,318480,60,5308.0,D,20,POPLATEK MESICNE,1997-08-18,10305.5,0.0,2584.0,97.0,2317.0,5307.5,1997-08-18,1998-10-12,878356.0,7572.034483,11817.595990,-1011.0,104762.0,39954.034483,23451.696220,447360.0,41655.0,389341.0,2303.0,59976.0,0.0,445057.0,371020.0,0.0,846920.0,1164.0,0.0,27804.0,165.0,2303.0,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542,OWNER,M,1952-08-26,No Card,NaT
4,4968,38,1998-04-19,110736,48,2307.0,C,19,POPLATEK TYDNE,1997-08-08,9666.8,0.0,231.0,228.0,6901.0,2306.8,1997-08-08,1998-04-10,259391.0,4716.200000,5740.513545,13841.0,55991.0,31383.581818,10950.723180,148404.0,5896.0,105091.0,1114.0,29671.0,131490.0,15800.0,81316.0,0.0,229641.0,912.0,0.0,27604.0,120.0,1114.0,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099,OWNER,F,1940-01-30,No Card,NaT
